In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
tf.__version__

NameError: name 'tf' is not defined

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

NameError: name 'ImageDataGenerator' is not defined

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

In [0]:
cnn = tf.keras.models.Sequential()

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [0]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [0]:
cnn.add(tf.keras.layers.Flatten())

In [0]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [0]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [0]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)